In [ ]:
## load news data from database into dataframe anc cluster with https://towardsdatascience.com/all-the-news-17fa34b52b9d

In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio

In [20]:
import re
import sys
import nltk
nltk.download('punkt')
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from collections import Counter
import time

[nltk_data] Error loading punkt/english.pickle: Package
[nltk_data]     'punkt/english.pickle' not found in index


In [16]:
## database connection
host = "localhost"
port = 5432
dbname = "postgres"
username = "postgres"
pwd = "postgres"
conn = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, username, pwd))

In [17]:
## get articles table into datframe
sql = "SELECT * FROM articles;"
articles = sqlio.read_sql_query(sql, conn) # type: ignore

/tmp/ipykernel_5686/2722885557.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  articles = sqlio.read_sql_query(sql, conn) # type: ignore


In [18]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   urlid              49 non-null     object        
 1   headline           45 non-null     object        
 2   content            49 non-null     object        
 3   authors            49 non-null     object        
 4   uploadtimestamp    38 non-null     datetime64[ns]
 5   imageurl           49 non-null     object        
 6   imagedescription   49 non-null     object        
 7   scrapingtimestamp  49 non-null     datetime64[ns]
 8   source             49 non-null     object        
 9   topic              49 non-null     object        
dtypes: datetime64[ns](2), object(8)
memory usage: 4.0+ KB


In [ ]:
## Plan:
## 1) stemming
## 2) create word matrix
## 3) TFIDF
## 4) Clustering

## 5) majority vote of category within clusters to be used as cluster name

In [26]:
## 1) stemming  -------------> re und sys importieren?
stemmer = PorterStemmer()
##tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
progress = 0 #for keeping track of where the function is

def stemm(x):
   end = time.time()
   dirty = word_tokenize(x)
   tokens = []
   for word in dirty:
      if word.strip('.') == '': #this deals with the bug
         pass
      elif re.search(r'\d{1,}', word): #getting rid of digits
         pass
      else:
         tokens.append(word.strip('.'))
   global start
   global progress
   tokens = pos_tag(tokens) #
   progress += 1
   stems = ' '.join(stemmer.stem(key.lower()) for key, value in  tokens if value != 'NNP') #getting rid of proper nouns
   end = time.time()
   sys.stdout.write('\r {} percent, {} position, {} per second '.format(str(float(progress / len(articles))), str(progress), (1 / (end - start)))) #lets us see how much time is left 
   start = time.time()
   return stems

In [27]:
start = time.time()
articles['stems'] = articles.content.apply(lambda x: stemm(x))

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/home/vscode/nltk_data'
    - '/usr/local/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
